In [1]:
#!pip install emoji --upgrade

## Name 
- 不使用 BeautifulSoup 轉碼 > 改: 將"&#數字;"內容刪除
- "&\amp;" 以 "&" 代替
- 刪除 \n
- 刪除 \r

In [2]:
def getName(p):
    p_name = p.get("name")
        
    #將"&#數字;"內容刪除
    p_name = re.sub('&#.*;?','', p_name)

    #p_name = BeautifulSoup(p_name, 'html.parser')
    #p_name = str(p_name)

    p_name = p_name.replace("&amp;", "&")
    p_name = p_name.replace("\n", "")
    p_name = p_name.replace("\r", "")
    
    return p_name

## Price
- $\$$ 5.99 > 把 $\$$ 拿掉
- $\$$ 8.99 - $\$$ 26.49 > 兩個相加取平均
- 出現See low price in cart：child_item取最後一個


In [3]:
def getPriceJSON(headers, p_id, store_id_and_key):
    url = 'https://redsky.target.com/web/pdp_location/v1/tcin/'+ p_id +'?'+ store_id_and_key
    req = requests.get(url, headers=headers)
    data = req.json()
    return data

In [4]:
def getPrice(p, headers, p_id, store_id_and_key):
    p_price = p.get("price")
    
    if "NA" in p.get("price"):
        data = getPriceJSON(headers, p_id, store_id_and_key)
        if data.get('price'):
            p['price'] = data['price']['formatted_current_price']
    
    #price 把 $ 拿掉
    if "$" in p.get("price"):
        p_price = p.get('price').replace('$', '').replace(',', '')
        
        #price 兩個相加取平均
        if " - " in p_price:
            p_price_rang = p_price.split(' - ')
            p_price = (float(p_price_rang[0]) + float(p_price_rang[1]))/2 
            p_price = round(p_price, 2)
        else:
            p_price = float(p_price)
    
    #出現See low price in cart：
    if p.get('price')=="See low price in cart":
        data = getPriceJSON(headers, p_id, store_id_and_key)
        #child_item取最後一個
        if data.get('child_items'):
            p_price = data['child_items'][-1]['price']['current_retail']
            p_price = float(p_price)
        #沒child_item找price current_retail
        elif data.get('price'):
            if data['price'].get('current_retail'):
                p_price = data['price']['current_retail']
                p_price = float(p_price)
            else:
                print('price - no current_retail in price', p.get('url'))
        else:
            print('price - no price & child_items: ', p.get('url'))
    
    return p_price

## Category
- 找不到 Breadcrumbs > 找 product_classification
- 15% off Select Baby Feeding & Accessories > 佔忽略不處理

In [5]:
def getMoreInfo(headers, p_id, store_id_and_key):
    
    url ='https://redsky.target.com/v3/pdp/tcin/'+ p_id 
    url +='?excludes=taxonomy%2Cbulk_ship%2Cawesome_shop%2Cquestion_answer_statistics%2Crating_and_review_reviews%2Crating_and_review_statistics%2Cdeep_red_labels%2Cin_store_location%2Cavailable_to_promise_store%2Cavailable_to_promise_network'
    url +='&'+store_id_and_key+'&fulfillment_test_mode=grocery_opu_team_member_test'
    
    #print('json_url:', url)
    req = requests.get(url, headers=headers)
    data = req.json()
    
    return data

In [6]:
def getCategory(headers, p_id, store_id_and_key):
    p_category = p.get("category")
    
    if (len(p.get("category")) == 0 or p.get("category")=='NA'):
        try:
            req = requests.get(p.get("url"))
            soup = BeautifulSoup(req.text, "html")
            p_category = soup.find("div", {"class", "h-text-sm h-padding-v-tiny"}).text.strip("\u200e").split("/")
            
            if (p_category[0]==''):
                #找 product_classification
                data = getMoreInfo(headers, p_id, store_id_and_key)
                p_class = data['product']['item']['product_classification']
                #append it
                p_category = []
                p_category.append(p_class['product_type_name'])
                p_category.append(p_class['item_type_name'])          
        except:
            print('category - no breadcrumbs & product_classification', p.get("url"))
    
    return p_category

## Star_ratings
- 將 list 轉為 tuple > .tuple()
- 將後面評分數改成 float

In [7]:
def getRating(p):
    p_rating = p.get("star_ratings")
    if p.get("star_ratings"):
        p["star_ratings"][1] = float(p["star_ratings"][1])
        p_rating = tuple(p["star_ratings"])
    return p_rating

## Highlights
- 刪除「\n」、「●」、「*」、「｜」
- 刪除(see nutrition information for Saturated Fat, and Sodium content)
- NA值不用處理
- 將\\&\\#39;替換為'

In [8]:
def getHlits(p):
    p_hlits = p.get("highlights")
    
    if (p.get("highlights")!='NA'):
        for e, text in enumerate (p.get("highlights")):
            # define desired replacements
            rep = {
                '*': "", 
                '\n': " ",
                '\r':"", 
                '●':"", 
                'see nutrition information for Saturated Fat, and Sodium content':'',
                '|':'', 
                '&#39;':"'"
            } 
            
            rep = dict((re.escape(k), v) for k, v in rep.items()) 
            pattern = re.compile("|".join(rep.keys()))
            new_text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
            p["highlights"][e] = new_text
            p_hlits = p["highlights"]
    
    return p_hlits

## Specifications
- NA值不用處理
- Package Quantity 轉integer
- Net Weight：把"單位"和"數值"存成一個tuple
- 去除每一項目前面的空白
- 單位統一小寫
- 沒有單位 > 單位改“NA”

In [9]:
def getSpec(p):
    p_spec = p.get("specifications")
    
    if (p.get("specifications")!='NA'):
        for k, v in p.get("specifications").items():
            #去除每一項目前面的空白
            p['specifications'][k] = v.strip()
            
            if (k=='Package Quantity'):
                #Package Quantity 轉integer
                p['specifications'][k] = int(v)
                
            if (k=='Net weight'):
                #Splitting text and number in string
                res = re.findall(r'(\d*.?\d*)( \w+ *.*)', v.strip())
                try:
                    lst = list(res[0])
                    lst[0] = float(lst[0])
                    #單位統一小寫
                    lst[1] = lst[1].strip().lower()
                    t = tuple(lst)
                    #Net Weight：把"單位"和"數值"存成一個tuple
                    p['specifications'][k] = t
                except IndexError:
                    #沒有單位 > 單位改“NA”
                    t = (float(v),'NA')
                    p['specifications'][k] = t
        p_spec = p['specifications']
    
    return p_spec 

## Description
- 刪除「*」、「｜」特殊字元
- \"al dente\"（斜線是跳脫字元符號）
- ½ 改成 half
- NA值不用處理
- 刪除「PACKAGING MAY VARY BY LOCATION」
- 用strip()刪除句子最前面或最後面的”-”


In [10]:
def getDesc(p):
    p_desc = p.get("description")
    
    if (p.get("description")!='NA'):
            text = p.get("description").strip()
            # define desired replacements
            rep = {
                '*': "", 
                '\n': " ",
                '●':"", 
                '½':'half',
                '|':'', 
                'PACKAGING MAY VARY BY LOCATION':"",
                'Packaging may vary by location.':'', 
                '\r':' '
            } 
            
            rep = dict((re.escape(k), v) for k, v in rep.items()) 
            pattern = re.compile("|".join(rep.keys()))
            p_desc = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    
    return p_desc

## Reviews
- 刪除「*」
- 刪除 「❁」
- 刪除 「🤔」
- 刪除[This review was collected as part of a promotion.]
- w/ 改成 with
- w/o 和 w/out 改成 without
- ❤️ 改成「love」（不管出現多少 ❤️ 都改成一個love）


In [11]:
def getReview(p):
    p_review = p.get("reviews") 
    
    if (p.get("reviews")!='NA'):
        for e, text in enumerate (p.get("reviews")):
            #decode emoji 
            text = emoji.demojize(text)
            
            #remove duplicate words
            lst = text.split()
            newlist = sorted(set(lst), key=lambda x:lst.index(x))
            text = " ".join(newlist)
            
            #刪除[... promotion.]
            text = re.sub('\[.*promotion\.\]','', text)
            
            # define desired replacements
            rep = {
                '*': "", 
                '�Ÿ˜ƒ':'',
                '❁':'',
                'w/':'with',
                'w/o':'without', 
                '<3':'love', 
                ':)':'smile', 
                ':-)':'smile',
                '=)':'smile',
                ';D':'smile',
                ':))':'smile',
                ';)':'smile',
                ':D':'smile',                  
            }
            
            rep = dict((re.escape(k), v) for k, v in rep.items()) 
            pattern = re.compile("|".join(rep.keys()))
            new_text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
            p["reviews"][e] = new_text
        
        p_review = p["reviews"]
    return p_review 

# Main.py
- store_id_and_key = '改成自己的'
- with open('改成自己的讀取json檔名', 'r', encoding="utf-8") as r:
- with open('改成自己的另存新json檔名', 'a', encoding='utf-8') as f:

In [12]:
import json
import html
from bs4 import BeautifulSoup
import requests
import re
import emoji
import os


with open('./product_info_LA1.json', 'r', encoding="utf-8") as r:
    c = r.read()
    r_list = json.loads(c)

#get url res
useragent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
headers = {'User-Agent':useragent}

#改成自己的store_id&key
store_id_and_key = 'pricing_store_id=1357&key=eb2551e4accc14f38cc42d32fbc2b2ea'

count = 0
# Remove a file
os.remove('../product_info_LA1_modify.json')
for p in r_list:
    p_id = p.get("url").split('A-')[1]
    

    p_name = getName(p)
    p['name'] = p_name
    
    p_price = getPrice(p, headers, p_id, store_id_and_key)
    p['price'] = p_price
    
    p_category = getCategory(headers, p_id, store_id_and_key)
    p['category'] = p_category
    
    p_rating = getRating(p)
    p["star_ratings"] = p_rating
    
    p_hlits = getHlits(p)
    p["highlights"]= p_hlits
    
    p_spec = getSpec(p)
    p['specifications'] = p_spec
    
    p_desc = getDesc(p)
    p['description'] = p_desc 
    
    p_review = getReview(p)
    p["reviews"]= p_review 
    
    with open('../product_info_LA1_modify.json', 'a', encoding='utf-8') as f: 
        json.dump(p, f, indent=4, ensure_ascii=False)
        f.write(',')
        count+=1
        print('%s record created successfully'%count)


1 record created successfully
2 record created successfully
3 record created successfully
4 record created successfully
5 record created successfully
6 record created successfully
7 record created successfully
8 record created successfully
9 record created successfully
10 record created successfully
11 record created successfully
12 record created successfully
13 record created successfully
14 record created successfully
15 record created successfully
16 record created successfully
17 record created successfully
18 record created successfully
19 record created successfully
20 record created successfully
21 record created successfully
22 record created successfully
23 record created successfully
24 record created successfully
25 record created successfully
26 record created successfully
27 record created successfully
28 record created successfully
29 record created successfully
30 record created successfully
31 record created successfully
32 record created successfully
33 record created

262 record created successfully
263 record created successfully
264 record created successfully
265 record created successfully
266 record created successfully
267 record created successfully
268 record created successfully
269 record created successfully
270 record created successfully
271 record created successfully
272 record created successfully
273 record created successfully
274 record created successfully
275 record created successfully
276 record created successfully
277 record created successfully
278 record created successfully
279 record created successfully
280 record created successfully
281 record created successfully
282 record created successfully
283 record created successfully
284 record created successfully
285 record created successfully
286 record created successfully
287 record created successfully
288 record created successfully
289 record created successfully
290 record created successfully
291 record created successfully
292 record created successfully
293 reco

543 record created successfully
544 record created successfully
545 record created successfully
546 record created successfully
547 record created successfully
548 record created successfully
549 record created successfully
550 record created successfully
551 record created successfully
552 record created successfully
553 record created successfully
554 record created successfully
555 record created successfully
556 record created successfully
557 record created successfully
558 record created successfully
559 record created successfully
560 record created successfully
561 record created successfully
562 record created successfully
563 record created successfully
564 record created successfully
565 record created successfully
566 record created successfully
567 record created successfully
568 record created successfully
569 record created successfully
570 record created successfully
571 record created successfully
572 record created successfully
573 record created successfully
574 reco

898 record created successfully
899 record created successfully
900 record created successfully
901 record created successfully
902 record created successfully
903 record created successfully
904 record created successfully
905 record created successfully
906 record created successfully
907 record created successfully
908 record created successfully
909 record created successfully
910 record created successfully
911 record created successfully
912 record created successfully
913 record created successfully
914 record created successfully
915 record created successfully
916 record created successfully
917 record created successfully
918 record created successfully
919 record created successfully
920 record created successfully
921 record created successfully
922 record created successfully
923 record created successfully
924 record created successfully
925 record created successfully
926 record created successfully
927 record created successfully
928 record created successfully
929 reco

1282 record created successfully
1283 record created successfully
1284 record created successfully
1285 record created successfully
1286 record created successfully
1287 record created successfully
1288 record created successfully
1289 record created successfully
1290 record created successfully
1291 record created successfully
1292 record created successfully
1293 record created successfully
1294 record created successfully
1295 record created successfully
1296 record created successfully
1297 record created successfully
1298 record created successfully
1299 record created successfully
1300 record created successfully
1301 record created successfully
1302 record created successfully
1303 record created successfully
1304 record created successfully
1305 record created successfully
1306 record created successfully
1307 record created successfully
1308 record created successfully
1309 record created successfully
1310 record created successfully
1311 record created successfully
1312 recor

...

46150 record created successfully
46151 record created successfully
46152 record created successfully
46153 record created successfully
46154 record created successfully
46155 record created successfully
46156 record created successfully
46157 record created successfully
46158 record created successfully
46159 record created successfully
46160 record created successfully
46161 record created successfully
46162 record created successfully
46163 record created successfully
46164 record created successfully
46165 record created successfully
46166 record created successfully
46167 record created successfully
46168 record created successfully
46169 record created successfully
46170 record created successfully
46171 record created successfully
46172 record created successfully
46173 record created successfully
46174 record created successfully
46175 record created successfully
46176 record created successfully
46177 record created successfully
46178 record created successfully
46179 record c

46391 record created successfully
46392 record created successfully
46393 record created successfully
46394 record created successfully
46395 record created successfully
46396 record created successfully
46397 record created successfully
46398 record created successfully
46399 record created successfully
46400 record created successfully
46401 record created successfully
46402 record created successfully
46403 record created successfully
46404 record created successfully
46405 record created successfully
46406 record created successfully
category - no breadcrumbs & product_classification https://www.target.com/p/hershey-s-milk-chocolate-bars-6ct/-/A-14708428
46407 record created successfully
46408 record created successfully
46409 record created successfully
46410 record created successfully
46411 record created successfully
46412 record created successfully
46413 record created successfully
46414 record created successfully
46415 record created successfully
46416 record created succes

46625 record created successfully
46626 record created successfully
46627 record created successfully
46628 record created successfully
46629 record created successfully
46630 record created successfully
46631 record created successfully
46632 record created successfully
46633 record created successfully
46634 record created successfully
46635 record created successfully
46636 record created successfully
46637 record created successfully
46638 record created successfully
46639 record created successfully
46640 record created successfully
46641 record created successfully
46642 record created successfully
46643 record created successfully
46644 record created successfully
46645 record created successfully
46646 record created successfully
46647 record created successfully
46648 record created successfully
46649 record created successfully
46650 record created successfully
46651 record created successfully
46652 record created successfully
46653 record created successfully
46654 record c

46866 record created successfully
46867 record created successfully
46868 record created successfully
46869 record created successfully
46870 record created successfully
46871 record created successfully
46872 record created successfully
46873 record created successfully
46874 record created successfully
46875 record created successfully
46876 record created successfully
46877 record created successfully
46878 record created successfully
46879 record created successfully
46880 record created successfully
46881 record created successfully
46882 record created successfully
46883 record created successfully
46884 record created successfully
46885 record created successfully
46886 record created successfully
46887 record created successfully
46888 record created successfully
46889 record created successfully
46890 record created successfully
46891 record created successfully
46892 record created successfully
46893 record created successfully
46894 record created successfully
46895 record c

47107 record created successfully
47108 record created successfully
47109 record created successfully
47110 record created successfully
47111 record created successfully
47112 record created successfully
47113 record created successfully
47114 record created successfully
47115 record created successfully
47116 record created successfully
47117 record created successfully
47118 record created successfully
47119 record created successfully
47120 record created successfully
47121 record created successfully
47122 record created successfully
47123 record created successfully
47124 record created successfully
47125 record created successfully
47126 record created successfully
47127 record created successfully
47128 record created successfully
47129 record created successfully
47130 record created successfully
47131 record created successfully
47132 record created successfully
47133 record created successfully
47134 record created successfully
47135 record created successfully
47136 record c

## 確認原檔案筆數
- with open('原黨名稱', 'r', encoding="utf-8") as r:

In [13]:
import json
with open('./product_info_LA1.json', 'r', encoding="utf-8") as r:
    c = r.read()
    r_list = json.loads(c)
    print(len(r_list))

47276


## 確認新檔案內容
- 檔案要前後加上[]
- with open('新黨案名稱', 'r', encoding='utf-8') as f:

In [15]:
import json
with open('../product_info_LA1_modify.json', 'r', encoding='utf-8') as f:
    new_list = json.loads(f.read())
    print(len(new_list))
    
#for np in new_list:
    #print(np.get("description"))     

47276
